In [1]:
import pandas as pd


full_data = pd.read_excel(r"D:\Downloads\DSTC vòng 3\raw data\VNINDEX.xlsx")
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 369736 entries, 0 to 369735
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  369736 non-null  int64  
 1   ticker      369736 non-null  object 
 2   timestamp   369736 non-null  object 
 3   open        369736 non-null  float64
 4   high        369736 non-null  float64
 5   low         369736 non-null  float64
 6   close       369736 non-null  float64
 7   volume      369736 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 22.6+ MB


In [2]:
from FiinQuantX import FiinSession
import pandas as pd
username = 'DSTC_28@fiinquant.vn'
password = 'Fiinquant0606'

client = FiinSession(username=username, password=password).login()
fi = client.FiinIndicator()

In [4]:
counts = full_data.groupby("ticker").size()
counts.describe()

count    412.000000
mean     897.417476
std      107.518076
min       87.000000
25%      919.000000
50%      919.000000
75%      919.000000
max      919.000000
dtype: float64

In [5]:
list_keep = counts[counts == 919].index
list_keep.size

373

In [6]:
df = full_data[full_data['ticker'].isin(list_keep)].reset_index()
df = df.drop("Unnamed: 0", axis = 1)
df = df.drop("index", axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 342787 entries, 0 to 342786
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ticker     342787 non-null  object 
 1   timestamp  342787 non-null  object 
 2   open       342787 non-null  float64
 3   high       342787 non-null  float64
 4   low        342787 non-null  float64
 5   close      342787 non-null  float64
 6   volume     342787 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 18.3+ MB


In [7]:
zero_counts_per_ticker = df["volume"].eq(0).groupby(df["ticker"]).sum()

# 2) Lấy 16 ticker có số lần volume=0 ít nhất
min_nonzero = zero_counts_per_ticker[zero_counts_per_ticker > 0]
tickers_min_16 = min_nonzero.nsmallest(16).index

df_zero_min16 = df[(df["ticker"].isin(tickers_min_16)) & (df["volume"] == 0)]

# 4) Lấy danh sách timestamp
timestamps_min16 = df_zero_min16["timestamp"].unique()


print("Timestamp mà 16 ticker ít volume=0 nhất có volume=0:")
print(timestamps_min16)

Timestamp mà 16 ticker ít volume=0 nhất có volume=0:
['2023-05-18 00:00' '2022-11-03 00:00' '2022-12-13 00:00'
 '2022-12-15 00:00' '2022-01-25 00:00' '2022-02-11 00:00'
 '2023-04-18 00:00' '2023-08-23 00:00' '2025-04-01 00:00'
 '2022-07-12 00:00' '2025-06-19 00:00' '2025-01-08 00:00'
 '2023-03-29 00:00' '2025-01-03 00:00' '2024-01-12 00:00'
 '2023-02-24 00:00' '2023-02-14 00:00' '2022-03-28 00:00'
 '2024-07-25 00:00' '2023-11-13 00:00']


In [8]:
df_filtered = df[~df["timestamp"].isin(timestamps_min16)]
df_filtered["timestamp"].nunique()

899

In [11]:
tickers_with_zero = df_filtered.loc[df_filtered['volume'] == 0, 'ticker'].unique()
print(tickers_with_zero.size)

115


In [12]:
df_clean = df_filtered[~df_filtered['ticker'].isin(tickers_with_zero)].copy()
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231942 entries, 1838 to 342786
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ticker     231942 non-null  object 
 1   timestamp  231942 non-null  object 
 2   open       231942 non-null  float64
 3   high       231942 non-null  float64
 4   low        231942 non-null  float64
 5   close      231942 non-null  float64
 6   volume     231942 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 14.2+ MB


In [13]:
print(df_clean['ticker'].unique())

['AAA' 'AAM' 'AAT' 'ABS' 'ACB' 'ACC' 'ACL' 'AGR' 'AGG' 'ANV' 'APG' 'APH'
 'HII' 'ASM' 'ASP' 'BAF' 'BCE' 'BCG' 'BFC' 'BIC' 'BID' 'BCM' 'DBD' 'BWE'
 'BKG' 'BMC' 'BMI' 'BMP' 'BSI' 'BVH' 'TNH' 'C47' 'CCL' 'CDC' 'CRE' 'STK'
 'CHP' 'CIG' 'CII' 'CKG' 'CLL' 'CMG' 'CMX' 'CNG' 'CRC' 'CSV' 'CTD' 'CTF'
 'CTG' 'CTI' 'ICT' 'CTS' 'D2D' 'DAH' 'ADS' 'DPG' 'DBC' 'DC4' 'DCL' 'DCM'
 'DGC' 'DGW' 'DHA' 'DHC' 'DHG' 'DHM' 'DIG' 'DLG' 'DPM' 'DPR' 'DQC' 'DRC'
 'DSN' 'DVP' 'DXG' 'DXS' 'FUESSV50' 'E1VFVN30' 'EIB' 'ELC' 'FUESSVFL'
 'EVE' 'EVG' 'FCM' 'FCN' 'FIR' 'FIT' 'FMC' 'FPT' 'FRT' 'FTS' 'FUEMAV30'
 'FUESSV30' 'FUEVFVND' 'FUEVN100' 'GAS' 'GDT' 'GEX' 'GIL' 'GEG' 'GMD'
 'GSP' 'HAG' 'HAH' 'HID' 'HAP' 'HAR' 'HAX' 'HCD' 'HCM' 'HDB' 'HDC' 'HDG'
 'HHP' 'HHS' 'TCH' 'HSL' 'HMC' 'HPG' 'HQC' 'HSG' 'HT1' 'HTN' 'HVH' 'HVX'
 'IDI' 'IJC' 'ITC' 'ITD' 'JVC' 'KBC' 'KDC' 'KDH' 'KHG' 'KHP' 'KMR' 'KOS'
 'KSB' 'LCG' 'LDG' 'LGL' 'LHG' 'LSS' 'LPB' 'MBB' 'MHC' 'MIG' 'MSB' 'MSN'
 'MWG' 'NAF' 'NBB' 'NCT' 'NHA' 'NHH' 'VHM' 'NKG' 'NLG' 'NN

In [14]:
df_clean = df_clean.reset_index()
df_clean = df_clean.drop("index", axis =1)


In [15]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231942 entries, 0 to 231941
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   ticker     231942 non-null  object 
 1   timestamp  231942 non-null  object 
 2   open       231942 non-null  float64
 3   high       231942 non-null  float64
 4   low        231942 non-null  float64
 5   close      231942 non-null  float64
 6   volume     231942 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 12.4+ MB


In [16]:
df_clean.tail(20)

,ticker,timestamp,open,high,low,close,volume
231922,YEG,2025-08-14 00:00,16350.0,16400.0,15550.0,15600.0,4970254
231923,YEG,2025-08-15 00:00,15600.0,15700.0,14650.0,14900.0,8061449
231924,YEG,2025-08-18 00:00,15000.0,15400.0,14800.0,15250.0,4842068
231925,YEG,2025-08-19 00:00,15400.0,16300.0,15200.0,16000.0,10069801
231926,YEG,2025-08-20 00:00,16150.0,16150.0,14950.0,15200.0,5784840
231927,YEG,2025-08-21 00:00,15300.0,15350.0,14850.0,14850.0,4251005
231928,YEG,2025-08-22 00:00,14600.0,14800.0,13850.0,13850.0,10601230
231929,YEG,2025-08-25 00:00,14000.0,14150.0,13550.0,13550.0,2886294
231930,YEG,2025-08-26 00:00,13550.0,14200.0,13550.0,14200.0,1986953
231931,YEG,2025-08-27 00:00,14200.0,14300.0,13950.0,14000.0,2561072


In [17]:
import numpy as np

tickers = df_clean['ticker'].unique()
all_dfs = []

    
for t in tickers:
    df_t = df_clean[df_clean['ticker'] == t].copy()

    df_t['ema_50'] = fi.ema(df_t['close'], window=50)
    df_t['ema_200'] = fi.ema(df_t['close'], window=200)
    df_t['macd'] = fi.macd(df_t['close'], window_fast=12, window_slow=26)
    df_t['macd_signal'] = fi.macd_signal(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
    df_t['macd_diff'] = fi.macd_diff(df_t['close'], window_fast=12, window_slow=26, window_sign=9)
    df_t['rsi'] = fi.rsi(df_t['close'], window=14) 
    df_t['bollinger_hband'] = fi.bollinger_hband(df_t['close'], window=20, window_dev=2)
    df_t['bollinger_lband'] = fi.bollinger_lband(df_t['close'], window=20, window_dev=2)
    df_t['mfi'] = fi.mfi(df_t['high'], df_t['low'], df_t['close'], df_t['volume'], window=14) 

    df_t['ema_200'] = np.where(df_t['ema_200'].isna(), df_t['ema_50'], df_t['ema_200'])

    all_dfs.append(df_t)



df_indicators = pd.concat(all_dfs, ignore_index=True)

print(df_indicators.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231942 entries, 0 to 231941
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ticker           231942 non-null  object 
 1   timestamp        231942 non-null  object 
 2   open             231942 non-null  float64
 3   high             231942 non-null  float64
 4   low              231942 non-null  float64
 5   close            231942 non-null  float64
 6   volume           231942 non-null  int64  
 7   ema_50           219300 non-null  float64
 8   ema_200          219300 non-null  float64
 9   macd             225492 non-null  float64
 10  macd_signal      223428 non-null  float64
 11  macd_diff        223428 non-null  float64
 12  rsi              228588 non-null  float64
 13  bollinger_hband  227040 non-null  float64
 14  bollinger_lband  227040 non-null  float64
 15  mfi              228536 non-null  object 
dtypes: float64(12), int64(1), object(3)
me

In [18]:
df_indicators.head(20)

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,macd_signal,macd_diff,rsi,bollinger_hband,bollinger_lband,mfi
0,AAA,2022-01-04 00:00,19302.030,20550.420,19302.030,19878.210,7354500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA,2022-01-05 00:00,19974.240,20934.540,19830.195,20118.285,7187400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA,2022-01-06 00:00,20070.270,21510.720,19974.240,21510.720,12375600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,2022-01-07 00:00,22038.885,22230.945,21654.765,21894.840,7009900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA,2022-01-10 00:00,22086.900,22471.020,20406.375,20406.375,11396800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AAA,2022-01-11 00:00,20454.390,20886.525,19206.000,19206.000,13383600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AAA,2022-01-12 00:00,19013.940,19686.150,17861.580,18821.880,12757700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AAA,2022-01-13 00:00,18821.880,19398.060,17525.475,17525.475,7501100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AAA,2022-01-14 00:00,17141.355,18245.700,16997.310,17621.505,7069100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AAA,2022-01-17 00:00,17717.535,17957.610,16421.130,16421.130,10039900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df_cleaned = df_indicators.dropna(how = 'any').copy().reset_index()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219248 entries, 0 to 219247
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            219248 non-null  int64  
 1   ticker           219248 non-null  object 
 2   timestamp        219248 non-null  object 
 3   open             219248 non-null  float64
 4   high             219248 non-null  float64
 5   low              219248 non-null  float64
 6   close            219248 non-null  float64
 7   volume           219248 non-null  int64  
 8   ema_50           219248 non-null  float64
 9   ema_200          219248 non-null  float64
 10  macd             219248 non-null  float64
 11  macd_signal      219248 non-null  float64
 12  macd_diff        219248 non-null  float64
 13  rsi              219248 non-null  float64
 14  bollinger_hband  219248 non-null  float64
 15  bollinger_lband  219248 non-null  float64
 16  mfi              219248 non-null  obje

In [20]:
df_cleaned = df_cleaned.drop("index", axis =1)
df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,macd_signal,macd_diff,rsi,bollinger_hband,bollinger_lband,mfi
0,AAA,2022-03-23 00:00,17093.34,17573.490,17045.325,17381.430,8173000,17553.936472,17553.936472,-92.902238,-153.782113,60.879875,53.245839,17657.024086,16486.442414,54.381724
1,AAA,2022-03-24 00:00,17381.43,18581.805,17285.400,18581.805,19729800,17594.245041,17594.245041,34.962480,-116.033194,150.995674,65.885556,18029.483680,16286.836820,63.295334
2,AAA,2022-03-25 00:00,18725.85,19109.970,18245.700,18341.730,10262600,17623.558177,17623.558177,115.591675,-69.708220,185.299896,62.260266,18231.988116,16209.171384,74.881069
3,AAA,2022-03-29 00:00,18245.70,19109.970,18149.670,18629.820,9976000,17663.019425,17663.019425,200.426952,-15.681186,216.108138,64.765706,18478.138178,16131.073822,82.9241
4,AAA,2022-03-30 00:00,18437.76,18917.910,17765.550,17861.580,9806100,17670.806114,17670.806114,203.325166,28.120084,175.205082,54.395216,18531.675478,16135.154522,78.506132


In [21]:
df_cleaned['golden_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['ema_50'] > x['ema_200']) & (x['ema_50'].shift(1) <= x['ema_200'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['death_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['ema_50'] < x['ema_200']) & (x['ema_50'].shift(1) >= x['ema_200'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['return'] = df_cleaned.groupby('ticker')['close'].pct_change()

df_cleaned['macd_cross'] = df_cleaned.groupby('ticker').apply(
    lambda x: ((x['macd'] > x['macd_signal']) & (x['macd'].shift(1) <= x['macd_signal'].shift(1))).astype(int) -
              ((x['macd'] < x['macd_signal']) & (x['macd'].shift(1) >= x['macd_signal'].shift(1))).astype(int)
).reset_index(level=0, drop=True)

df_cleaned['bollinger_pct'] = (df_cleaned['close'] - df_cleaned['bollinger_lband']) / (df_cleaned['bollinger_hband'] - df_cleaned['bollinger_lband'])

df_cleaned['bollinger_bw'] = (df_cleaned['bollinger_hband'] - df_cleaned['bollinger_lband']) / ((df_cleaned['bollinger_hband'] + df_cleaned['bollinger_lband']) / 2)


df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,...,rsi,bollinger_hband,bollinger_lband,mfi,golden_cross,death_cross,return,macd_cross,bollinger_pct,bollinger_bw
0,AAA,2022-03-23 00:00,17093.34,17573.490,17045.325,17381.430,8173000,17553.936472,17553.936472,-92.902238,...,53.245839,17657.024086,16486.442414,54.381724,0,0,NaN,0,0.764567,0.068568
1,AAA,2022-03-24 00:00,17381.43,18581.805,17285.400,18581.805,19729800,17594.245041,17594.245041,34.962480,...,65.885556,18029.483680,16286.836820,63.295334,0,0,0.069061,0,1.316944,0.101564
2,AAA,2022-03-25 00:00,18725.85,19109.970,18245.700,18341.730,10262600,17623.558177,17623.558177,115.591675,...,62.260266,18231.988116,16209.171384,74.881069,0,0,-0.012920,0,1.054252,0.117465
3,AAA,2022-03-29 00:00,18245.70,19109.970,18149.670,18629.820,9976000,17663.019425,17663.019425,200.426952,...,64.765706,18478.138178,16131.073822,82.9241,0,0,0.015707,0,1.064626,0.135632
4,AAA,2022-03-30 00:00,18437.76,18917.910,17765.550,17861.580,9806100,17670.806114,17670.806114,203.325166,...,54.395216,18531.675478,16135.154522,78.506132,0,0,-0.041237,0,0.720388,0.138260


In [22]:
df_cleaned.dropna(inplace=True)
df_cleaned = df_cleaned.reset_index()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218990 entries, 0 to 218989
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   index            218990 non-null  int64  
 1   ticker           218990 non-null  object 
 2   timestamp        218990 non-null  object 
 3   open             218990 non-null  float64
 4   high             218990 non-null  float64
 5   low              218990 non-null  float64
 6   close            218990 non-null  float64
 7   volume           218990 non-null  int64  
 8   ema_50           218990 non-null  float64
 9   ema_200          218990 non-null  float64
 10  macd             218990 non-null  float64
 11  macd_signal      218990 non-null  float64
 12  macd_diff        218990 non-null  float64
 13  rsi              218990 non-null  float64
 14  bollinger_hband  218990 non-null  float64
 15  bollinger_lband  218990 non-null  float64
 16  mfi              218990 non-null  obje

In [23]:
df_cleaned.columns

Index(['index', 'ticker', 'timestamp', 'open', 'high', 'low', 'close',
       'volume', 'ema_50', 'ema_200', 'macd', 'macd_signal', 'macd_diff',
       'rsi', 'bollinger_hband', 'bollinger_lband', 'mfi', 'golden_cross',
       'death_cross', 'return', 'macd_cross', 'bollinger_pct', 'bollinger_bw'],
      dtype='object')

In [24]:
df_cleaned = df_cleaned.drop("index", axis =1)

In [25]:
df_cleaned.head()

,ticker,timestamp,open,high,low,close,volume,ema_50,ema_200,macd,...,rsi,bollinger_hband,bollinger_lband,mfi,golden_cross,death_cross,return,macd_cross,bollinger_pct,bollinger_bw
0,AAA,2022-03-24 00:00,17381.430,18581.805,17285.400,18581.805,19729800,17594.245041,17594.245041,34.962480,...,65.885556,18029.483680,16286.836820,63.295334,0,0,0.069061,0,1.316944,0.101564
1,AAA,2022-03-25 00:00,18725.850,19109.970,18245.700,18341.730,10262600,17623.558177,17623.558177,115.591675,...,62.260266,18231.988116,16209.171384,74.881069,0,0,-0.012920,0,1.054252,0.117465
2,AAA,2022-03-29 00:00,18245.700,19109.970,18149.670,18629.820,9976000,17663.019425,17663.019425,200.426952,...,64.765706,18478.138178,16131.073822,82.9241,0,0,0.015707,0,1.064626,0.135632
3,AAA,2022-03-30 00:00,18437.760,18917.910,17765.550,17861.580,9806100,17670.806114,17670.806114,203.325166,...,54.395216,18531.675478,16135.154522,78.506132,0,0,-0.041237,0,0.720388,0.138260
4,AAA,2022-03-31 00:00,18005.625,18437.760,17813.565,17957.610,5252700,17682.053325,17682.053325,210.939264,...,55.357487,18597.945829,16169.715671,73.165823,0,0,0.005376,0,0.736295,0.139683


In [26]:
df_cleaned.to_excel(r"cleaned data\VNINDEX_cleaned.xlsx")